In [9]:
!pip install optuna
import optuna
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# train 데이터 확인
df_train = pd.read_csv('/content/drive/MyDrive/DACON/전기차 가격 예측 해커톤: 데이터로 EV를 읽다!/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/DACON/전기차 가격 예측 해커톤: 데이터로 EV를 읽다!/test.csv')
df_train

,ID,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
0,TRAIN_0000,P사,TayGTS,Nearly New,86.077,AWD,13642,0,No,2,159.66
1,TRAIN_0001,K사,Niro,Nearly New,56.000,FWD,10199,6,No,0,28.01
2,TRAIN_0002,A사,eT,Brand New,91.200,AWD,2361,7,No,0,66.27
3,TRAIN_0003,A사,RSeTGT,Nearly New,NaN,AWD,21683,3,No,0,99.16
4,TRAIN_0004,B사,i5,Pre-Owned,61.018,AWD,178205,1,No,0,62.02
...,...,...,...,...,...,...,...,...,...,...,...
7492,TRAIN_7492,H사,ION5,Brand New,NaN,AWD,3773,10,No,0,35.95
7493,TRAIN_7493,B사,i3,Pre-Owned,46.000,RWD,135411,2,No,0,23.40
7494,TRAIN_7494,P사,TayCT,Brand New,NaN,AWD,1363,2,No,0,120.00
7495,TRAIN_7495,B사,i3,Nearly New,56.000,RWD,39445,6,No,2,24.00


In [3]:
# train 데이터 정보 확인
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        7497 non-null   object 
 1   제조사       7497 non-null   object 
 2   모델        7497 non-null   object 
 3   차량상태      7497 non-null   object 
 4   배터리용량     4786 non-null   float64
 5   구동방식      7497 non-null   object 
 6   주행거리(km)  7497 non-null   int64  
 7   보증기간(년)   7497 non-null   int64  
 8   사고이력      7497 non-null   object 
 9   연식(년)     7497 non-null   int64  
 10  가격(백만원)   7497 non-null   float64
dtypes: float64(2), int64(3), object(6)
memory usage: 644.4+ KB


In [4]:
# test 데이터 정보 확인
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        846 non-null    object 
 1   제조사       846 non-null    object 
 2   모델        846 non-null    object 
 3   차량상태      846 non-null    object 
 4   배터리용량     542 non-null    float64
 5   구동방식      846 non-null    object 
 6   주행거리(km)  846 non-null    int64  
 7   보증기간(년)   846 non-null    int64  
 8   사고이력      846 non-null    object 
 9   연식(년)     846 non-null    int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 66.2+ KB


In [5]:
# 각 데이터에 있는 결측치를 평균으로 대체
df_train.fillna(df_train['배터리용량'].mean(), inplace = True)
df_test.fillna(df_test['배터리용량'].mean(), inplace = True)

In [6]:
# train 데이터 중 ID열 제거
df_train.drop(['ID'], axis = 1, inplace = True)
df_train

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
0,P사,TayGTS,Nearly New,86.077000,AWD,13642,0,No,2,159.66
1,K사,Niro,Nearly New,56.000000,FWD,10199,6,No,0,28.01
2,A사,eT,Brand New,91.200000,AWD,2361,7,No,0,66.27
3,A사,RSeTGT,Nearly New,69.397187,AWD,21683,3,No,0,99.16
4,B사,i5,Pre-Owned,61.018000,AWD,178205,1,No,0,62.02
...,...,...,...,...,...,...,...,...,...,...
7492,H사,ION5,Brand New,69.397187,AWD,3773,10,No,0,35.95
7493,B사,i3,Pre-Owned,46.000000,RWD,135411,2,No,0,23.40
7494,P사,TayCT,Brand New,69.397187,AWD,1363,2,No,0,120.00
7495,B사,i3,Nearly New,56.000000,RWD,39445,6,No,2,24.00


In [7]:
# train 데이터 중 문자형 데이터를 LabelEncoding 변환
train_column_list = list(df_train.columns)
train_column_list = train_column_list[0:3] + train_column_list[4:5] + train_column_list[7:8]

for i in train_column_list:
  encoder = LabelEncoder()
  encoder.fit(df_train[i])
  df_train[i] = encoder.transform(df_train[i])

df_train

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
0,4,16,1,86.077000,0,13642,0,0,2,159.66
1,3,10,1,56.000000,1,10199,6,0,0,28.01
2,0,17,0,91.200000,0,2361,7,0,0,66.27
3,0,12,1,69.397187,0,21683,3,0,0,99.16
4,1,19,2,61.018000,0,178205,1,0,0,62.02
...,...,...,...,...,...,...,...,...,...,...
7492,2,2,0,69.397187,0,3773,10,0,0,35.95
7493,1,18,2,46.000000,2,135411,2,0,0,23.40
7494,4,15,0,69.397187,0,1363,2,0,0,120.00
7495,1,18,1,56.000000,2,39445,6,0,2,24.00


In [13]:
def prediction_model(trial):
  # x, y 정의
  x = df_train.drop(columns = ['가격(백만원)'])
  y = df_train['가격(백만원)']

  # 데이터 분할 및 모델학습
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

  # Optuna에 사용할 하이퍼파라미터 설정
  params = {
      'objective':'reg:squarederror',
      'eval_metric':'rmse',
      'n_estimators':trial.suggest_int('n_estimators', 100, 3000),
      'learning_rate':trial.suggest_float('learning_rate', 1e-9, 1e-1, log = True),
      'max_depth':trial.suggest_int('max_depth', 3, 15),
      'reg_alpha':trial.suggest_float('reg_alpha', 0.0, 1.0),
      'reg_lambda':trial.suggest_float('reg_lambda', 0.0, 10.0),
      'random_state':trial.suggest_int('random_state', 0, 100)
  }

  # XGBRegressor 객체 생성 및 학습
  xgb_rgs = XGBRegressor(**params)
  xgb_rgs.fit(x_train, y_train)

  # 예측값 도출
  y_pred = xgb_rgs.predict(x_test)
  mse = mean_squared_error(y_test, y_pred)
  rmse = np.sqrt(mse)

  return rmse

In [14]:
# prediction_model의 최적 하이퍼파라미터 찾기
study = optuna.create_study(direction = 'minimize')
study.optimize(prediction_model, n_trials = 300)

[I 2025-01-15 05:07:35,109] A new study created in memory with name: no-name-9f483d98-28e6-4c28-b90a-a6ea6e18033b
[I 2025-01-15 05:07:38,678] Trial 0 finished with value: 1.5655584021337943 and parameters: {'n_estimators': 1543, 'learning_rate': 0.013557334230980647, 'max_depth': 9, 'reg_alpha': 0.8660467894216788, 'reg_lambda': 6.770011409655923, 'random_state': 33}. Best is trial 0 with value: 1.5655584021337943.
[I 2025-01-15 05:07:38,846] Trial 1 finished with value: 36.943941241568325 and parameters: {'n_estimators': 123, 'learning_rate': 2.9248499033537836e-05, 'max_depth': 7, 'reg_alpha': 0.01821123996344065, 'reg_lambda': 9.052767316425506, 'random_state': 89}. Best is trial 0 with value: 1.5655584021337943.
[I 2025-01-15 05:07:41,361] Trial 2 finished with value: 33.83510220090997 and parameters: {'n_estimators': 1688, 'learning_rate': 5.654403237716036e-05, 'max_depth': 12, 'reg_alpha': 0.6750981278816389, 'reg_lambda': 9.435639484067888, 'random_state': 8}. Best is trial 0 w

In [22]:
# 최적 하이퍼파라미터
optimum_params = study.best_params
print(optimum_params)

{'n_estimators': 2983, 'learning_rate': 0.002515166637963296, 'max_depth': 6, 'reg_alpha': 0.856141075181666, 'reg_lambda': 9.995557836355868, 'random_state': 2}


In [16]:
# test 데이터 확인
df_test

,ID,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년)
0,TEST_000,P사,TayCT,Nearly New,76.093000,AWD,14057,2,No,0
1,TEST_001,B사,iX,Brand New,90.000000,AWD,7547,8,No,0
2,TEST_002,B사,i5,Brand New,69.712963,RWD,7197,7,Yes,0
3,TEST_003,H사,ION5,Nearly New,68.479000,AWD,10357,7,No,1
4,TEST_004,K사,EV6,Brand New,69.712963,FWD,7597,10,No,0
...,...,...,...,...,...,...,...,...,...,...
841,TEST_841,P사,TayGTS,Pre-Owned,64.683000,AWD,117298,2,No,0
842,TEST_842,V사,ID4,Pre-Owned,55.547000,AWD,72308,0,No,0
843,TEST_843,V사,ID4,Pre-Owned,55.547000,AWD,124537,0,No,0
844,TEST_844,A사,Q4eT,Nearly New,69.646000,AWD,15629,4,No,0


In [17]:
# test 데이터 중 문자형 데이터를 LabelEncoding 변환
test_column_list = list(df_test.columns)
test_column_list = test_column_list[1:4] + test_column_list[5:6] + test_column_list[8:9]

for i in test_column_list:
  encoder = LabelEncoder()
  encoder.fit(df_test[i])
  df_test[i] = encoder.transform(df_test[i])

df_test

,ID,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년)
0,TEST_000,4,15,1,76.093000,0,14057,2,0,0
1,TEST_001,1,20,0,90.000000,0,7547,8,0,0
2,TEST_002,1,19,0,69.712963,2,7197,7,1,0
3,TEST_003,2,2,1,68.479000,0,10357,7,0,1
4,TEST_004,3,0,0,69.712963,1,7597,10,0,0
...,...,...,...,...,...,...,...,...,...,...
841,TEST_841,4,16,2,64.683000,0,117298,2,0,0
842,TEST_842,6,1,2,55.547000,0,72308,0,0,0
843,TEST_843,6,1,2,55.547000,0,124537,0,0,0
844,TEST_844,0,11,1,69.646000,0,15629,4,0,0


In [23]:
# 최적 하이퍼파라미터를 이용한 객체 생성
xgb_rgs = XGBRegressor(**optimum_params)

# train 데이터 분할 및 모델학습
x = df_train.drop(columns = ['가격(백만원)'])
y = df_train['가격(백만원)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

xgb_rgs.fit(x_train, y_train)

# 학습된 모델을 통해 test 데이터 예측가격 도출
x = df_test.drop(columns = ['ID'])
y_price = xgb_rgs.predict(x)

df_trial = pd.DataFrame({'ID':df_test['ID'], '가격(백만원)':y_price})
df_trial

,ID,가격(백만원)
0,TEST_000,129.955261
1,TEST_001,79.966751
2,TEST_002,64.633568
3,TEST_003,35.225178
4,TEST_004,47.316353
...,...,...
841,TEST_841,151.421890
842,TEST_842,38.894558
843,TEST_843,38.894558
844,TEST_844,58.834133


In [24]:
# 제출 파일 생성
df_trial.to_csv('/content/drive/MyDrive/DACON/전기차 가격 예측 해커톤: 데이터로 EV를 읽다!/submission_0115.csv', index = False)